# 01_scrap

## Getting data from The Reporters

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import json

In [2]:
url = 'https://www.thereporters.co/page/'

In [3]:
bs = []
page = 1

while page <= 2:
    r = requests.get(url + str(page) + '/?s=ส่งออกทุเรียน')
    s = BeautifulSoup(r.content, 'html.parser')
    bs.append(s)
    page += 1
r

<Response [200]>

In [4]:
title = []
content = []
source_url = []
publish_date = []
retrieve_date = []

In [5]:
for i in range(len(bs)):
    for t in bs[i].find_all('div', {'class':'entry-blog-list-right'}):
        links = t.find('a')['href']
        source_url.append(links)
# print(len(source_url))

In [6]:
for u in source_url:
    news_url = requests.get(u)
    bs_news = BeautifulSoup(news_url.content, 'html.parser')
    
    for j in bs_news.find_all('div', {'class':'blog-header'}):
        topic = j.find('h1').text
        topic = topic.strip()
        topic = topic.replace('\n', '')
        title.append(topic)
        
        publish = j.find('time')['datetime']
        publish = publish.split('T')[0]
        publish_date.append(publish)
        
        retrieve = str(datetime.date.today())
        retrieve_date.append(retrieve)
        
    for k in bs_news.find_all('div', {'class':'single-entry-summary-post-content'}):
        con = k.text
        con = con.strip()
        con = con.replace('\n', ' ')
        con = con.replace('\xa0', ' ')
        content.append(con)

# print(content)

In [7]:
table = pd.DataFrame([title, content, source_url, publish_date, retrieve_date]).transpose()
table.columns = ['title', 'content', 'source_url', 'publish_date', 'retrieve_date']
table.head()

,title,content,source_url,publish_date,retrieve_date
0,นายกฯ ยินดีส่งออกทุเรียนไทยไปจีนทะลุเกินเป้า 5...,นายธนกร วังบุญคงชนะ โฆษกประจำสำนักนายกรัฐมนตรี...,https://www.thereporters.co/business/2406221021/,2022-06-24,2022-11-06
1,“จุรินทร์” เปิดงาน “พาณิชย์ Fruit Festival 202...,วันนี้ (2 พ.ค. 65) ณ ลานหน้าศูนย์การค้าเซ็นทรั...,https://www.thereporters.co/tw-politics/020522...,2022-05-02,2022-11-06
2,ก.เกษตรฯ ดีเดย์ 22 เม.ย.นี้ ‘Big ATK Day’ ฆ่าเ...,นายอลงกรณ์ พลบุตร ที่ปรึกษารัฐมนตรีว่าการกระทร...,https://www.thereporters.co/tw-politics/180422...,2022-04-18,2022-11-06
3,‘ทุเรียนไทย’ กุญแจนำทางยักษ์ใหญ่วงการ ‘ผลไม้’ ...,ฉงชิ่ง – เทศบาลนครฉงชิ่งทางตะวันตกเฉียงใต้ของจ...,https://www.thereporters.co/tw-food/290920222118/,2022-09-29,2022-11-06
4,อลงกรณ์”เผยจีนเปิดด่านรถไฟผิงเสียงขนทุเรียนไทย...,นายอลงกรณ์ พลบุตร ที่ปรึกษารัฐมนตรีว่าการกระทร...,https://www.thereporters.co/tw-politics/290422...,2022-04-29,2022-11-06


In [8]:
table_1 = table.to_dict('records')
print(f"Total news : {len(table_1)}")

Total news : 13


In [9]:
with open('exporting_durians_the_reporters.json', 'w') as f:
    json.dump(table_1, f)

## Upload data to DataPlatform

In [10]:
from dsmlibrary.datanode import DataNode

In [11]:
dir_raw_id = 60

In [12]:
datanode = DataNode()

Init DataNode sucessful!


In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='exporting_durians_the_reporters.json')